# Programa Ciencia de los Datos
## Mineria de Datos e Inteligencia de Negocios
### Tarea #3 - Prof. Lorena Zuñiga
### Estudiante: Sergio Castillo

## CRISP-DM
### Objetivo:
Se requiere identificar caracteristicas propias de las personas que las lleven a tener ingresos superiores a los $50 mil anuales.

### Definicion del problema:
Que caracteristicas de las personas son las que influencian principalmente su comportamiento y las llevan a obtener mejores ingresos con respecto a otras personas?
### Supuestos

Se tomara un dataset que contiene caracterisiticas de una poblacion de 25 mil personas, el cual ha sido anonimizado pero que contiene los siguientes rubros:
Edad, tipo de trabajo, nivel educativo, años educativos, estado civil, ocupacion, sexo, horas laborales semanalmente, pais de origen e ingresos.

Basado en este dataset se tratara de identificar patrones comunes entre las personas cuyos ingresos superan los $50 mil. De modo tal que se puedan definir reglas de negocio lo mas precisas posibles que intenten predecir los ingresos de una persona a partir de las caracteristicas que vaya demostrando.
### Criterio de exito
El estudio sera considerada exitoso si se logra determinar al menos 3 reglas determinantes que aplicadas a las personas les ayuden a mejorar sus ingresos.

In [150]:
#Cargar librerias necesarias
install.packages("ggpubr")
install.packages("arules")
install.packages("arulesViz")
library(ggpubr)
library(arules)
library(arulesViz)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [115]:
#Cargar dataset
data = read.csv("AdultosUSA.csv", sep=";", header = TRUE, stringsAsFactors = FALSE)

In [116]:
str(data)

'data.frame':	25000 obs. of  10 variables:
 $ Edad          : int  39 50 38 53 28 37 49 52 31 42 ...
 $ TipoTrabajo   : chr  "State-gov" "Self-emp-not-inc" "Private" "Private" ...
 $ NivelEducativo: chr  "Bachelors" "Bachelors" "HS-grad" "11th" ...
 $ AnnosEducacion: int  13 13 9 7 13 14 5 9 14 13 ...
 $ EstadoCivil   : chr  "Never-married" "Married-civ-spouse" "Divorced" "Married-civ-spouse" ...
 $ Ocupacion     : chr  "Adm-clerical" "Exec-managerial" "Handlers-cleaners" "Handlers-cleaners" ...
 $ Sexo          : chr  "Male" "Male" "Male" "Male" ...
 $ HorasSemanales: int  40 13 40 40 40 40 16 45 50 40 ...
 $ PaisOrigen    : chr  "United-States" "United-States" "United-States" "United-States" ...
 $ Ingresos      : chr  "<=50K." "<=50K." "<=50K." "<=50K." ...


#### Años de educacion y nive educativo parecen ser redundantes
### Exploracion de datos

In [117]:
table(data$TipoTrabajo)
gghistogram(data, x = "TipoTrabajo", y = "..count..", stat="count", x.text.angle = 45, width = 1)
print("Contiene NA:")
anyNA(data$TipoTrabajo)
#Vemos que el tipo de trabajo hay muchos con valores con signo de pregunta, pero no hay ningun valor nulo


               ?      Federal-gov        Local-gov     Never-worked 
            1399              750             1624                5 
         Private     Self-emp-inc Self-emp-not-inc        State-gov 
           17385              857             1978              993 
     Without-pay 
               9 

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”Warning message:
“Ignoring unknown parameters: binwidth, bins, pad”

[1] "Contiene NA:"


[1] FALSE

In [71]:
#Revisamos los valores para otra columna y si hay nulos de una vez
table(data$NivelEducativo)
print("Contiene NA:")
anyNA(data$NivelEducativo)
gghistogram(data, x = "NivelEducativo", y = "..count..", stat="count",x.text.angle = 45, width = 1)


        10th         11th         12th      1st-4th      5th-6th      7th-8th 
         721          909          323          120          244          491 
         9th   Assoc-acdm    Assoc-voc    Bachelors    Doctorate      HS-grad 
         394          801         1059         4140          315         8120 
     Masters    Preschool  Prof-school Some-college 
        1300           36          430         5597 

[1] "Contiene NA:"


[1] FALSE

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”Warning message:
“Ignoring unknown parameters: binwidth, bins, pad”

#### El nivel educativo es mejor guia sobre el factor de escolaridad que los años de educacion, por lo que sera removida del estudio.

In [72]:
table(data$EstadoCivil)
print("Contiene NA:")
anyNA(data$EstadoCivil)
gghistogram(data, x = "EstadoCivil", y = "..count..", stat="count", x.text.angle = 45, width = 1)


             Divorced     Married-AF-spouse    Married-civ-spouse 
                 3435                    16                 11441 
Married-spouse-absent         Never-married             Separated 
                  328                  8225                   786 
              Widowed 
                  769 

[1] "Contiene NA:"


[1] FALSE

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”Warning message:
“Ignoring unknown parameters: binwidth, bins, pad”

In [73]:
table(data$Ocupacion)
print("Contiene NA:")
anyNA(data$Ocupacion)
gghistogram(data, x = "Ocupacion", y = "..count..", stat="count", x.text.angle = 45, width = 1)
#Tambien hay valores desconocidos pero no nulos


                ?      Adm-clerical      Armed-Forces      Craft-repair 
             1404              2975                 7              3122 
  Exec-managerial   Farming-fishing Handlers-cleaners Machine-op-inspct 
             3084               767              1000              1536 
    Other-service   Priv-house-serv    Prof-specialty   Protective-serv 
             2555               121              3180               503 
            Sales      Tech-support  Transport-moving 
             2815               703              1228 

[1] "Contiene NA:"


[1] FALSE

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”Warning message:
“Ignoring unknown parameters: binwidth, bins, pad”

In [149]:
table(data$PaisOrigen)
print("Contiene NA:")
anyNA(data$PaisOrigen)
gghistogram(data, x = "PaisOrigen", y = "..count..", stat="count", x.text.angle = 45, width = 1)
#Tambien hay valores deconocidos para el pais de origen


                         ?                   Cambodia 
                       445                         16 
                    Canada                      China 
                        99                         60 
                  Columbia                       Cuba 
                        45                         72 
        Dominican-Republic                    Ecuador 
                        54                         19 
               El-Salvador                    England 
                        72                         72 
                    France                    Germany 
                        21                        102 
                    Greece                  Guatemala 
                        23                         50 
                     Haiti         Holand-Netherlands 
                        38                          1 
                  Honduras                       Hong 
                         8                         11 
         

[1] "Contiene NA:"


[1] FALSE

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”Warning message:
“Ignoring unknown parameters: binwidth, bins, pad”

#### Del total de 25 mil muestras el pais origen siendo Estados Unidos contiene 22421 lo que equivale al 89% de la poblacion, esta relacion claramente no corresponde a una distribucion normal acertada por lo que se decide que este atributo sera eliminado del estudio.

In [76]:
table(data$Ingresos)
print("Contiene NA:")
anyNA(data$Ingresos)
gghistogram(data, x = "Ingresos", y = "..count..", stat="count", x.text.angle = 45, width = 1)


<=50K.  >50K. 
 19016   5984 

[1] "Contiene NA:"


[1] FALSE

Warning message:
“Using `bins = 30` by default. Pick better value with the argument `bins`.”Warning message:
“Ignoring unknown parameters: binwidth, bins, pad”

### Calidad de datos
Se logra verificar que casi todas las columnas contienen valores validos y no nulos, unicamente las columnas PaisOrigen, Ocupacion y TipoTrabajo contiene valores desconocidos marcados con el simbolo de pregunta "?" pero no se cree que interfieran con el estudio, se utilizaran todas las columnas para el estudio excepto PaisOrigen pues ensuciaria la muestra y AnnosEducacion pues es redundante con NivelEducativo y la ultima es una mejor guia para el estudio.

### Seleccion de Datos / Limpieza / Transformacion a factores
La limpieza incluye la eliminacion de columnas consideradas no necesarias para el estudio, asi como la conversion a factores y transacciones.

In [143]:
data2 = data
data2$TipoTrabajo = as.factor(data2$TipoTrabajo)
data2$NivelEducativo = as.factor(data2$NivelEducativo)
data2$AnnosEducacion = NULL
data2$EstadoCivil = as.factor(data2$EstadoCivil)
data2$Ocupacion = as.factor(data2$Ocupacion)
data2$Sexo = as.factor(data2$Sexo)
data2$HorasSemanales = as.factor(data2$HorasSemanales)
data2$PaisOrigen = NULL
data2$Ingresos = as.factor(data2$Ingresos)
str(data2)

'data.frame':	25000 obs. of  8 variables:
 $ Edad          : int  39 50 38 53 28 37 49 52 31 42 ...
 $ TipoTrabajo   : Factor w/ 9 levels "?","Federal-gov",..: 8 7 5 5 5 5 5 7 5 5 ...
 $ NivelEducativo: Factor w/ 16 levels "10th","11th",..: 10 10 12 2 10 13 7 12 13 10 ...
 $ EstadoCivil   : Factor w/ 7 levels "Divorced","Married-AF-spouse",..: 5 3 1 3 3 3 4 3 5 3 ...
 $ Ocupacion     : Factor w/ 15 levels "?","Adm-clerical",..: 2 5 7 7 11 5 9 5 11 5 ...
 $ Sexo          : Factor w/ 2 levels "Female","Male": 2 2 2 2 1 1 1 2 1 2 ...
 $ HorasSemanales: Factor w/ 94 levels "1","2","3","4",..: 40 13 40 40 40 40 16 45 50 40 ...
 $ Ingresos      : Factor w/ 2 levels "<=50K.",">50K.": 1 1 1 1 1 1 1 2 2 2 ...


In [144]:
#Convertimos el data frame a transactions
trns = as(data2, "transactions")
str(trns)

Warning message:
“Column(s) 1 not logical or factor. Applying default discretization (see '? discretizeDF').”

Formal class 'transactions' [package "arules"] with 3 slots
  ..@ data       :Formal class 'ngCMatrix' [package "Matrix"] with 5 slots
  .. .. ..@ i       : int [1:200000] 1 10 21 32 36 51 91 146 2 9 ...
  .. .. ..@ p       : int [1:25001] 0 8 16 24 32 40 48 56 64 72 ...
  .. .. ..@ Dim     : int [1:2] 148 25000
  .. .. ..@ Dimnames:List of 2
  .. .. .. ..$ : NULL
  .. .. .. ..$ : NULL
  .. .. ..@ factors : list()
  ..@ itemInfo   :'data.frame':	148 obs. of  3 variables:
  .. ..$ labels   : chr [1:148] "Edad=[17,31)" "Edad=[31,44)" "Edad=[44,90]" "TipoTrabajo=?" ...
  .. ..$ variables: Factor w/ 8 levels "Edad","EstadoCivil",..: 1 1 1 8 8 8 8 8 8 8 ...
  .. ..$ levels   : Factor w/ 147 levels "?","[17,31)",..: 2 3 4 1 118 122 130 134 139 140 ...
  ..@ itemsetInfo:'data.frame':	25000 obs. of  1 variable:
  .. ..$ transactionID: chr [1:25000] "1" "2" "3" "4" ...


#### Para este estudio no se ve necesaria la generacion de nuevos atributos ni registros generados.

In [145]:
summary(trns)

transactions as itemMatrix in sparse format with
 25000 rows (elements/itemsets/transactions) and
 148 columns (items) and a density of 0.054 

most frequent items:
               Ingresos=<=50K.            TipoTrabajo=Private 
                         19016                          17385 
                     Sexo=Male              HorasSemanales=40 
                         16709                          11713 
EstadoCivil=Married-civ-spouse                        (Other) 
                         11441                         123736 

element (itemset/transaction) length distribution:
sizes
    8 
25000 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      8       8       8       8       8       8 

includes extended item information - examples:
        labels variables  levels
1 Edad=[17,31)      Edad [17,31)
2 Edad=[31,44)      Edad [31,44)
3 Edad=[44,90]      Edad [44,90]

includes extended transaction information - examples:
  transactionID
1             1
2             2
3  

### Seleccion de tecnica de modelado
Se utilizara la funcion apriori con el objetivo de generar las reglas de asociacion basada en los labels identificados por medio de las transacciones.
La idea sera obtener una lista de reglas que tengan al menos un 50% de confianza y elegir las 3 mejores.

In [146]:
#Listar los labels identificados en las transacciones
trns@itemInfo$labels

[1] "Edad=[17,31)"                      "Edad=[31,44)"                     
  [3] "Edad=[44,90]"                      "TipoTrabajo=?"                    
  [5] "TipoTrabajo=Federal-gov"           "TipoTrabajo=Local-gov"            
  [7] "TipoTrabajo=Never-worked"          "TipoTrabajo=Private"              
  [9] "TipoTrabajo=Self-emp-inc"          "TipoTrabajo=Self-emp-not-inc"     
 [11] "TipoTrabajo=State-gov"             "TipoTrabajo=Without-pay"          
 [13] "NivelEducativo=10th"               "NivelEducativo=11th"              
 [15] "NivelEducativo=12th"               "NivelEducativo=1st-4th"           
 [17] "NivelEducativo=5th-6th"            "NivelEducativo=7th-8th"           
 [19] "NivelEducativo=9th"                "NivelEducativo=Assoc-acdm"        
 [21] "NivelEducativo=Assoc-voc"          "NivelEducativo=Bachelors"         
 [23] "NivelEducativo=Doctorate"          "NivelEducativo=HS-grad"           
 [25] "NivelEducativo=Masters"            "NivelEducativo=Preschool"         
 [27] "NivelEducativo=Prof-school"        "NivelEducativo=Some-college"      
 [29] "EstadoCivil=Divorced"              "EstadoCivil=Married-AF-spouse"    
 [31] "EstadoCivil=Married-civ-spouse"    "EstadoCivil=Married-spouse-absent"
 [33] "EstadoCivil=Never-married"         "EstadoCivil=Separated"            
 [35] "EstadoCivil=Widowed"               "Ocupacion=?"                      
 [37] "Ocupacion=Adm-clerical"            "Ocupacion=Armed-Forces"           
 [39] "Ocupacion=Craft-repair"            "Ocupacion=Exec-managerial"        
 [41] "Ocupacion=Farming-fishing"         "Ocupacion=Handlers-cleaners"      
 [43] "Ocupacion=Machine-op-inspct"       "Ocupacion=Other-service"          
 [45] "Ocupacion=Priv-house-serv"         "Ocupacion=Prof-specialty"         
 [47] "Ocupacion=Protective-serv"         "Ocupacion=Sales"                  
 [49] "Ocupacion=Tech-support"            "Ocupacion=Transport-moving"       
 [51] "Sexo=Female"                       "Sexo=Male"                        
 [53] "HorasSemanales=1"                  "HorasSemanales=2"                 
 [55] "HorasSemanales=3"                  "HorasSemanales=4"                 
 [57] "HorasSemanales=5"                  "HorasSemanales=6"                 
 [59] "HorasSemanales=7"                  "HorasSemanales=8"                 
 [61] "HorasSemanales=9"                  "HorasSemanales=10"                
 [63] "HorasSemanales=11"                 "HorasSemanales=12"                
 [65] "HorasSemanales=13"                 "HorasSemanales=14"                
 [67] "HorasSemanales=15"                 "HorasSemanales=16"                
 [69] "HorasSemanales=17"                 "HorasSemanales=18"                
 [71] "HorasSemanales=19"                 "HorasSemanales=20"                
 [73] "HorasSemanales=21"                 "HorasSemanales=22"                
 [75] "HorasSemanales=23"                 "HorasSemanales=24"                
 [77] "HorasSemanales=25"                 "HorasSemanales=26"                
 [79] "HorasSemanales=27"                 "HorasSemanales=28"                
 [81] "HorasSemanales=29"                 "HorasSemanales=30"                
 [83] "HorasSemanales=31"                 "HorasSemanales=32"                
 [85] "HorasSemanales=33"                 "HorasSemanales=34"                
 [87] "HorasSemanales=35"                 "HorasSemanales=36"                
 [89] "HorasSemanales=37"                 "HorasSemanales=38"                
 [91] "HorasSemanales=39"                 "HorasSemanales=40"                
 [93] "HorasSemanales=41"                 "HorasSemanales=42"                
 [95] "HorasSemanales=43"                 "HorasSemanales=44"                
 [97] "HorasSemanales=45"                 "HorasSemanales=46"                
 [99] "HorasSemanales=47"                 "HorasSemanales=48"                
[101] "HorasSemanales=49"                 "HorasSemanales=50"                
[103] "HorasSemanales=51

### Construccion del modelo / Seleccion de Parametros / Ejecucion
Basado en los labels anteriores las reglas que buscamos deben tener como resultado a la derecha el valor 'Ingresos=>50K.'
Las reglas generadas se busca tengan un nivel de confianza de al menos el 50% con un nivel de soporte del 0.5%.
El nivel de confianza implica cuantas veces la resolucion x -> y resulta cierta en el dataset, mientras que el nivel de soporte implica que tran frecuentemente son vistos los valores en el dataset.

In [147]:
rules = apriori(trns, parameter = list(supp = 0.005, conf = 0.50, minlen = 2), appearance = list(default = "lhs", rhs = 'Ingresos=>50K.'))
summary(rules)

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5   0.005      2
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 125 

set item appearances ...[1 item(s)] done [0.00s].
set transactions ...[148 item(s), 25000 transaction(s)] done [0.05s].
sorting and recoding items ... [69 item(s)] done [0.01s].
creating transaction tree ... done [0.04s].
checking subsets of size 1 2 3 4 5 6 7 done [0.12s].
writing ... [423 rule(s)] done [0.00s].
creating S4 object  ... done [0.02s].


set of 423 rules

rule length distribution (lhs + rhs):sizes
  2   3   4   5   6   7 
  4  61 160 141  51   6 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    2.0     4.0     4.0     4.5     5.0     7.0 

summary of quality measures:
    support        confidence        lift         count     
 Min.   :0.005   Min.   :0.50   Min.   :2.1   Min.   : 125  
 1st Qu.:0.006   1st Qu.:0.56   1st Qu.:2.3   1st Qu.: 157  
 Median :0.009   Median :0.66   Median :2.8   Median : 215  
 Mean   :0.012   Mean   :0.66   Mean   :2.8   Mean   : 298  
 3rd Qu.:0.014   3rd Qu.:0.73   3rd Qu.:3.1   3rd Qu.: 349  
 Max.   :0.095   Max.   :0.91   Max.   :3.8   Max.   :2384  

mining info:
 data ntransactions support confidence
 trns         25000   0.005        0.5

### Visualizacion de reglas
A continuacion una visualizacion grafica de las 423 reglas obtenidas. Como podemos ver a mayor confianza menor nivel de soporte, lo que genera reglas casi exclusivas en el sentido de que si se cumplen sus condiciones, que suelen ser raras, entonces la prediccion sera cierta.

In [151]:
plot(rules)

To reduce overplotting, jitter is added! Use jitter = 0 to prevent jitter.


El grafico a continuacion solo representa cuantos atributos son considerados por regla, como vemos 5 atributos parece ser la distribucion mas comun que determina el resultado buscado.

In [152]:
plot(rules, method = "two-key plot")

To reduce overplotting, jitter is added! Use jitter = 0 to prevent jitter.


### Seleccion de reglas / Descripcion
Para la seleccion de las reglas se utilizara como parametro principal el nivel de confianza obtenido, el nivel de confianza en este caso viene muy de la mano con el "lift" que implica la proporcion de la relacion de los atributos con el resultado, en este caso podemos ver que la regla mas confiable es casi 4 veces mas probable de que se cumpla cuando el tipo de trabajo es privado, el nivel educativo es maestria, el estado civil es casado, la ocupacion es administracion ejecutiva y el sexo de la persona es hombre.

In [157]:
#Ordenamos las reglas por factor de confianza obtenido para ver las 10 mas relevantes
options(digits = 2)
rulesSorted = sort(rules, by="confidence", decreasing=TRUE)
inspect(rulesSorted[1:10])

     lhs                                 rhs              support confidence lift count
[1]  {TipoTrabajo=Private,                                                             
      NivelEducativo=Masters,                                                          
      EstadoCivil=Married-civ-spouse,                                                  
      Ocupacion=Exec-managerial,                                                       
      Sexo=Male}                      => {Ingresos=>50K.}  0.0057       0.91  3.8   142
[2]  {TipoTrabajo=Private,                                                             
      NivelEducativo=Masters,                                                          
      EstadoCivil=Married-civ-spouse,                                                  
      Ocupacion=Exec-managerial}      => {Ingresos=>50K.}  0.0059       0.90  3.7   147
[3]  {NivelEducativo=Masters,                                                          
      EstadoCivil=Married-civ-sp

#### Las 3 reglas seleccionadas literalmente del estudio son:
1. Si la persona trabaja en la empresa privada, tiene una maestria, esta casada y tiene un puesto ejecutivo-administrativo practicamente de seguro tendra un salario superior a los $50 mil.
2. Si la persona es mayor a 30 años, su nivel educativo es minimo un grado profesional y su puesto es grado profesional especializado tambien tiene muy altas probabilidades de tener un salario superior.
3. Si la persona trabaja en la empresa privada, tiene un nivel educativo grado profesional, tiene un puesto ejecutivo-administrativo y es varon tendra muy altas posibilidades de tener un salario superior.